## QuantumFinance - Parte Extra: BERT como Extrator de Features + ML Clássico

### Objetivo

Este notebook explora uma abordagem alternativa ao fine-tuning do BERT: utilizamos embeddings gerados por um modelo transformer pré-treinado para representar os textos e treinamos modelos de classificação tradicionais (como Regressão Logística e Random Forest) sobre esses embeddings.

---


In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sentence_transformers import SentenceTransformer
from sklearn import preprocessing

In [3]:
DEV_MODE = True
TEST_SIZE = 0.25
RANDOM_STATE = 42

df = pd.read_csv('tickets_reclamacoes_classificados.csv', sep=';')
if DEV_MODE:
    df = df.sample(frac=0.1, random_state=RANDOM_STATE).reset_index(drop=True)

In [5]:
#normalizacao 

import re
import string

def remove_punctuation(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def normalize_text(text):
    institution_names = ['chase', 'bank', 'jp', 'gm', 'financial', 'jpmcb']
    text = text.lower()
    text = re.sub(r'\d+|/', '', text)
    text = re.sub(r'\bx\b|\w*xx+\w*', '', text)
    text = remove_punctuation(text)
    text = re.sub(r'\s+', ' ', text).strip()
    for inst in institution_names:
        text = re.sub(r'\b' + re.escape(inst) + r'\b', '[INST]', text)
    return text

df['texto_limpo'] = df['descricao_reclamacao'].apply(normalize_text)
df

,id_reclamacao,data_abertura,categoria,descricao_reclamacao,texto_limpo
0,2505290,2017-06-08T12:00:00-05:00,Serviços de conta bancária,Oxxxx xxxx pagou o JP Morgan Chase {$ 1200.00}...,pagou o [INST] morgan [INST] para pagamento de...
1,3625355,2020-04-27T12:00:00-05:00,Roubo / Relatório de disputa,Perdi meu cartão de crédito (uma reserva de sa...,perdi meu cartão de crédito uma reserva de saf...
2,1405636,2015-06-04T12:00:00-05:00,Hipotecas / Empréstimos,"Em 2005, fui para xxxx xxxx xxxx. Eles me dire...",em fui para eles me direcionaram ao banco de u...
3,2915672,2018-05-23T12:00:00-05:00,Roubo / Relatório de disputa,Eu me inscrevi e recebi um cartão de crédito p...,eu me inscrevi e recebi um cartão de crédito p...
4,1733697,2016-01-08T12:00:00-05:00,Roubo / Relatório de disputa,"Com o Chase A {$ 620,00} Charge (xxxx 2015) fo...",com o [INST] a charge foi originalmente contes...
...,...,...,...,...,...
2102,1592842,2015-10-05T12:00:00-05:00,Serviços de conta bancária,"JP Morgan Chase Bank, xxxx xxxx xxxx, xxxx xxx...",[INST] morgan [INST] [INST] fl eu tenho uma co...
2103,4182434,2021-03-04T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Eu arquivei uma disputa em relação a itens inc...,eu arquivei uma disputa em relação a itens inc...
2104,1542982,2015-08-28T12:00:00-05:00,Outros,Eu tenho uma propriedade secundária na qual co...,eu tenho uma propriedade secundária na qual co...
2105,2682483,2017-09-23T12:00:00-05:00,Serviços de conta bancária,Recebi uma grande quantia de dinheiro como pre...,recebi uma grande quantia de dinheiro como pre...


In [6]:
#encoder 

label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['categoria'])

In [8]:
## Split dos Dados (Treino/Teste)

X_train, X_test, y_train, y_test = train_test_split(
    df['texto_limpo'], df['label'], test_size=TEST_SIZE,
    random_state=RANDOM_STATE, stratify=df['label']
)

In [9]:
## Geração dos Embeddings com SentenceTransformer


print("Carregando modelo de embeddings...")
model = SentenceTransformer('neuralmind/bert-base-portuguese-cased', device='cuda' if torch.cuda.is_available() else 'cpu')

print("Gerando embeddings de treino e teste...")
X_train_embeddings = model.encode(X_train.tolist(), show_progress_bar=True)
X_test_embeddings = model.encode(X_test.tolist(), show_progress_bar=True)

Carregando modelo de embeddings...


No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.


Gerando embeddings de treino e teste...


Batches:   0%|          | 0/50 [00:00<?, ?it/s]

Batches:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
## Treinamento e Avaliação com Modelos Clássicos

modelos = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'RandomForest': RandomForestClassifier(),
    'LinearSVC': LinearSVC(),
    'GradientBoosting': GradientBoostingClassifier(),
    'GaussianNB': GaussianNB()
}

print("\nResultados:")
for nome, modelo in modelos.items():
    try:
        modelo.fit(X_train_embeddings, y_train)
        y_pred = modelo.predict(X_test_embeddings)
        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f'{nome:<20} → F1 Score: {f1:.2%}')
        print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    except Exception as e:
        print(f'{nome:<20} → Erro: {e}')


Resultados:
LogisticRegression   → F1 Score: 77.20%
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.77      0.76      0.76       126
            Hipotecas / Empréstimos       0.81      0.78      0.80        88
                             Outros       0.82      0.62      0.71        53
       Roubo / Relatório de disputa       0.68      0.74      0.71       125
         Serviços de conta bancária       0.83      0.86      0.84       135

                           accuracy                           0.77       527
                          macro avg       0.78      0.75      0.77       527
                       weighted avg       0.78      0.77      0.77       527

RandomForest         → F1 Score: 66.85%
                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.67      0.73      0.70       126
            Hipotecas / Empréstimos       0.75      0.69